<table class="ee-notebook-buttons" align="left">
    <td><a target="_blank"  href="https://github.com/giswqs/geemap/tree/master/examples/notebooks/oil_palm_and_rubber_plantation.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
    <td><a target="_blank"  href="https://nbviewer.jupyter.org/github/giswqs/geemap/blob/master/examples/notebooks/oil_palm_and_rubber_plantation.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_blank"  href="https://colab.research.google.com/github/giswqs/geemap/blob/master/examples/notebooks/oil_palm_and_rubber_plantation.ipynb"><img width=26px src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

# Oil palm concessions in southern Myanmar consist mostly of unconverted forest

Credits to [Keiko Nomura](https://github.com/nkeikon)

GEE App: https://nkeikon.users.earthengine.app/view/tanintharyi-oil-palm-and-rubber-map

Source code: https://github.com/nkeikon/tanintharyi

Reference: https://go.nature.com/2QBIrYe

Demo: https://i.imgur.com/VrYKmVG.gifv

## Install Earth Engine API and geemap
Install the [Earth Engine Python API](https://developers.google.com/earth-engine/python_install) and [geemap](https://github.com/giswqs/geemap). The **geemap** Python package is built upon the [ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet) and [folium](https://github.com/python-visualization/folium) packages and implements several methods for interacting with Earth Engine data layers, such as `Map.addLayer()`, `Map.setCenter()`, and `Map.centerObject()`.
The following script checks if the geemap package has been installed. If not, it will install geemap, which automatically installs its [dependencies](https://github.com/giswqs/geemap#dependencies), including earthengine-api, folium, and ipyleaflet.

In [ ]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

In [ ]:
import ee
import geemap

## Create an interactive map 
The default basemap is `Google Maps`. [Additional basemaps](https://github.com/giswqs/geemap/blob/master/geemap/geemap.py#L13) can be added using the `Map.add_basemap()` function. 

In [ ]:
Map = geemap.Map(center=[40,-100], zoom=4)
Map.add_basemap('HYBRID') # Add Google Map
Map

## Add Earth Engine Python script 

In [ ]:
# Add Earth Engine data
area1 = ee.Image("users/nkeikon/myanmar_sr/area1_map")
area2 = ee.Image("users/nkeikon/myanmar_sr/area2_map")
tanintharyi = ee.FeatureCollection("users/nkeikon/myanmar_sr/TNI")
Map.centerObject(tanintharyi,10)
total = ee.ImageCollection([area1,area2]).mosaic()
palette =['ff0000',# palm  (red)
              '9933ff',#rubber  (purple)
              '008000',#other trees  (green)
              'lime',#shrub (lime)
              'yellow',#bare (yellow)
              '0000ff',#river (blue)
]
viz = {'min':1, 'max':6, 'palette':palette}
MAIN = 'Default map'
OILPALM = 'Oil palm only'
RUBBER = 'Rubber only'
OILPALM_RUBBER = 'Oil palm and rubber'
mainVis = total.visualize(**viz)
oilpalmVis = total.eq(1).selfMask().visualize(**{'palette':'red'})
rubberVis = total.eq(2).selfMask().visualize(**{'palette':'purple'})
oilpalm_rubberVis = oilpalmVis.blend(rubberVis)
Map.addLayer(mainVis, {}, 'Land cover')
Map.addLayer(oilpalmVis, {}, 'Oil palm only')
Map.addLayer(rubberVis, {}, 'Rubber only')
Map.addLayer(oilpalm_rubberVis, {}, 'Oil palm and rubber')

In [ ]:
# Add widgets to the map
import ipywidgets as widgets
from ipyleaflet import WidgetControl
legend = widgets.HTML(
    value='<img src="https://i.imgur.com/Ye2sgan.png">',
    placeholder='Land cover',
    descripition='Land cover'
)

citation = widgets.HTML(
    value='Reference: <a href="https://go.nature.com/2QBIrYe">https://go.nature.com/2QBIrYe</a>',
    placeholder='Land cover',
    descripition='Land cover'
)

dropdown = widgets.Dropdown(
    options=[('Land cover', 1), ('Old palm only', 2), ('Rubber only', 3), ('Oil palm and bubber only', 4), ('Show all layers', 5)],
    value=5,
    description='Select layer',
)

legend_control = WidgetControl(widget=legend, position='bottomleft')
citation_control = WidgetControl(widget=citation, position='bottomright')
dropdown_control = WidgetControl(widget=dropdown, position='topright')

all_layers = Map.layers

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        if change['new'] == 5: 
            Map.layers = all_layers
        else:
            Map.layers = Map.layers[:3] 
            layer_index = change['new'] + 2
            Map.add_layer(all_layers[layer_index])

dropdown.observe(on_change)

Map.add_control(legend_control)
Map.add_control(citation_control)
Map.add_control(dropdown_control)

## Display Earth Engine data layers 

In [ ]:
Map